In [1]:
#-------------------------YOLO-------------------------------------------------------
from __future__ import print_function
import numpy as np
import time
import cv2
import os
import pandas as pd
from os.path import isfile, join
from numba import jit
import os
import numpy as np
import matplotlib

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io
import glob
import time
import argparse




In [2]:
#Yolo weights paths
labelsPath = os.path.join('C:/Users/DELL/Desktop/CODES_PFE/YOLO_pretrained/coco.names')
LABELS = open(labelsPath).read().strip().split("\n")


In [3]:
#derive the paths to the YOLO weights and model configuration
weightsPath = os.path.join('C:/Users/DELL/Desktop/CODES_PFE/YOLO_pretrained/yolov3.weights')
configPath = os.path.join('C:/Users/DELL/Desktop/CODES_PFE/YOLO_pretrained/yolov3.cfg')
np.random.seed(42)
#initialize a list of colors to represent each possible class label
COLORS = np.random.randint(0, 255, size=(len(LABELS),3),dtype="uint8")
print("[INFO] Start loading YOLO from disk...")
network = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
print('[INFO] YOLO model Loaded .')
layer_names = network.getLayerNames()
outputs_layer_names = [ layer_names[i[0] - 1] for i in network.getUnconnectedOutLayers()]
success = 1
confidence_threshold = 0.9 #Minimum probability to filter weak detections
non_maxima_threshold = 0.3  #non-maxima suppression threshold
j=0
kp = []
f=0
num_frames = 143;
start = time.time()
cap = cv2.VideoCapture("C:/Users/DELL/Desktop/CODES_PFE/DATA_videos/action3.mp4")
(W, H) = (None, None)
frame_id = 0     
#df: dataframe when i put YOLO information
df = pd.DataFrame({ 'frame_id':[],'x': [], 'y': [], 'width':[],'height':[] })


[INFO] Start loading YOLO from disk...
[INFO] YOLO model Loaded .


In [4]:
#start loop
while True:
    (retrieved, frame) = cap.read()
    if (retrieved!=True):
        break
    if W is None or H is None:
        (H, W) = frame.shape[:2] 
    top_left_y = int ((H / 2) + (H / 4))
    bottom_right_x = int ((W / 3) * 2)
    bottom_right_y = int ((H / 2) - (H / 4))
        
#blobFromImage: facilitate image preprocessing for deep learning classification (Mean subtraction & scaling)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    network.setInput(blob)
    start = time.time() # grap the start time of the forward pass
    layerOutputs = network.forward(outputs_layer_names)
    end = time.time() # grap the end time of the forward pass

    print("[INFO] YOLO took {:.4f} seconds to trait frame {}".format((end - start) , frame_id )) 
    boxes = []
    confidences = []
    classIDs = []
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence
            if j == 0 or j == f+1:
                # the current object detection
                scores = detection[5:] # get all the scores
                classID = np.argmax(scores)# get the index of the max score

                confidence = scores[classID] # get the max score (probability)

                if confidence > confidence_threshold:

                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                   
                    
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        if j == 0 or j == f+1:
            indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold , non_maxima_threshold)
            if len(indices) > 0:
                for i in indices.flatten():
                    (x, y) = (boxes[i][0], boxes[i][1])
                    (w, h) = (boxes[i][2], boxes[i][3])
                    color = [int(c) for c in COLORS[classIDs[i]]]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)   
                    text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
                    
                    cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    df = df.append({'frame_id':frame_id,
                                'x': boxes[i][0], 
                                'y': boxes[i][1],
                                'width':boxes[i][2],
                                'height':boxes[i][3],
                                'type':LABELS[classIDs[i]],
                                'confidende':confidences[i]},
                                ignore_index=True)
                  

    if j == 0 :
        cv2.imwrite("C:/Users/DELL/Desktop/CODES_PFE/yolo_frames/frames%d.jpg" % j,frame)

    else:
        oldimage = cv2.imread("C:/Users/DELL/Desktop/CODES_PFE/yolo_frames/frames%d.jpg" % f)  
        newimage = frame
        if oldimage.shape == newimage.shape:
            difference = cv2.subtract(oldimage, newimage)
            b, g, r = cv2.split(difference)
            print(cv2.countNonZero(b))
            if cv2.countNonZero(b) < 10000 and cv2.countNonZero(g) < 10000 and cv2.countNonZero(r) <10000:
                print("image %d is Equal to %d" %(j,f))
            else :
                cv2.imwrite("C:/Users/DELL/Desktop/CODES_PFE/yolo_frames/frames%d.jpg" % j,frame)
                print("image %d is not Equal to image %d" %(f,j))
                print("image %d is being printed" %f)
                f=j
  
        #cv2.imwrite("C:/Users/DELL/Desktop/PFE/keyframe3/test%d.jpg" % j,frame)
    j+=1
    k = cv2.waitKey(30) & 0xff
    if k == ord('q'):
        break
    frame_id = frame_id +1
            # release the file pointer

print("[INFO] cleaning up...")
cap.release()
cv2.destroyAllWindows()
df = df.drop_duplicates()
df.to_csv('YOLO_information.csv')

[INFO] YOLO took 1.8416 seconds to trait frame 0
[INFO] YOLO took 1.3105 seconds to trait frame 1
210605
image 0 is not Equal to image 1
image 0 is being printed
[INFO] YOLO took 1.3294 seconds to trait frame 2
237955
image 1 is not Equal to image 2
image 1 is being printed
[INFO] YOLO took 1.0572 seconds to trait frame 3
236975
image 2 is not Equal to image 3
image 2 is being printed
[INFO] YOLO took 1.0193 seconds to trait frame 4
272039
image 3 is not Equal to image 4
image 3 is being printed
[INFO] YOLO took 0.9815 seconds to trait frame 5
225387
image 4 is not Equal to image 5
image 4 is being printed
[INFO] YOLO took 0.9538 seconds to trait frame 6
255367
image 5 is not Equal to image 6
image 5 is being printed
[INFO] YOLO took 1.1001 seconds to trait frame 7
229575
image 6 is not Equal to image 7
image 6 is being printed


KeyboardInterrupt: 

In [35]:
df = df.drop_duplicates()
df.to_csv('DATA_csv/YOLO_information.csv')